# Learning of Process Representations Using Recurrent Neural Networks

In [21]:
import numpy as np
import pandas as pd

from replearn.eventlog import EventLog

from replearn.embedding_predict import EmbeddingPredict
from replearn.autoencoder import AutoencoderRepresentation
from replearn.doc2vec import Doc2VecRepresentation

from replearn.clustering import Clustering
from replearn.evaluation import Evaluation

## Load event log

In [22]:
# event log configuration
event_log_path = '../logs/multi-perspective_02'
file_name = 'p2p_500_10_20_5_2_1-0.3-1.json.gz'

case_attributes = None # auto-detect attributes
event_attributes = ['concept:name', 'user'] # use activity name and user
true_cluster_label = 'cluster'

# load file
event_log = EventLog(file_name, case_attributes=case_attributes, event_attributes=event_attributes, true_cluster_label=true_cluster_label)
event_log.load(event_log_path + '/' + file_name, False)
event_log.preprocess()

## Representation Learning

In [23]:
# hyperparameters
n_epochs = 10
n_batch_size = 64
n_clusters = 5

vector_size = 32

### LSTM / GRU

In [24]:
# init and train LSTM
predictor = EmbeddingPredict(event_log)
predictor.build_model(embedding_dim=vector_size, gru_dim=vector_size, rnn='LSTM')
predictor.fit(epochs=n_epochs, batch_size=n_batch_size, verbose=True)

NotImplementedError: Cannot convert a symbolic Tensor (lstm_5/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
# get feature vector
pred_model, feature_vector, embedding_vector = predictor.predict()

### Clustering

In [ ]:
# cluster feature vector
cluster_analysis = Clustering(event_log)
cluster_analysis.cluster(feature_vector, 'agglomerative', n_clusters, 'cosine')

cluster_result = cluster_analysis.evaluate()

In [ ]:
print('Adjusted Rand Index: ' + str(cluster_result[0]))
print('Normalized Mutual Information: ' + str(cluster_result[1]))
print('F1-BCubed: ' + str(cluster_result[2]))

In [ ]:
evaluation_a = Evaluation(event_log)
evaluation_a.evaluate_clusters(n_clusters,cluster_analysis._pred_labels)

In [ ]:
import gzip
import json

with gzip.open('../logs/multi-perspective_02/huge_500_10_20_5_2_1-0.1-1.json.gz', "r") as f:
   data = f.read()
   j = json.loads (data.decode('utf-8'))
   print(json.dumps(j, indent=4, sort_keys=True))
